In [62]:
# some imports
import math
import numpy as np
import pandas as pd
from math import atan2
from collections import namedtuple
from circle_fit import hyper_fit

In [63]:
# if you have ros installed on your os, you can use rosbag_pandas to import the bag and convert it to a pandas dataframe
# https://github.com/eurogroep/rosbag_pandas

# import rosbag_pandas
# df_include = rosbag_pandas.bag_to_dataframe('samples/first_test/2020-01-15-10-54-28.bag', include=['/scan'])

In [123]:
class Parameters():
    def __str__(self):
        return '\n'.join(name + ": " + str(getattr(self, name))
                         for name in self.names)

# parameters
parameters = Parameters()
parameters.barrier_size_realtive = 0.1
parameters.usable_laser_range = 220.0

# what does delta_time do?
delta_time = 0
# laser_scan csv file to import
data = pd.read_csv("samples/first_test/bagfile-_scan.csv")
# data point to select form csv file
selecetdDatapoint = 42
# last fieldnumber of data row
lastField = 1080

In [124]:
# convert dataframe to the original format
row = data.loc[selecetdDatapoint]
row.ranges = []
row.intensities = []
for i in range(lastField+1):
    row.ranges.append(row["ranges%s" % i])
    row.intensities.append(row["intensities%s" % i])

scan_message = row

In [126]:
# We need to define our circle

Point = namedtuple("Point", ["x", "y"])

class Circle():
    def __init__(self, center, radius):
        self.center = center
        self.radius = radius

    def create_array(self, start_angle, end_angle, sample_count=50):
        points = np.zeros((sample_count, 2))
        angles = np.linspace(start_angle, end_angle, sample_count)
        points[:, 0] = self.center.x + np.sin(angles) * self.radius
        points[:, 1] = self.center.y + np.cos(angles) * self.radius
        return points

    def get_angle(self, point):
        return atan2(point.x - self.center.x, point.y - self.center.y)

    def get_closest_point(self, point):
        x = point.x - self.center.x
        y = point.y - self.center.y
        distance = (x**2 + y**2) ** 0.5
        return Point(
            self.center.x + x * self.radius / distance,
            self.center.y + y * self.radius / distance
        )

    @staticmethod
    def fit(points):
        center_x, center_y, radius, _ = hyper_fit(points)
        return Circle(Point(center_x, center_y), radius)

In [3]:
def get_scan_as_cartesian(laser_scan):
    ranges = np.array(laser_scan.ranges)

    angles = np.linspace(
        laser_scan.angle_min,
        laser_scan.angle_max,
        ranges.shape[0])

    laser_range = laser_scan.angle_max - laser_scan.angle_min
    usable_range = math.radians(parameters.usable_laser_range)
    if usable_range < laser_range:
        skip_left = int((-laser_scan.angle_min - usable_range / 2) / laser_range * ranges.shape[0])
        skip_right = int((laser_scan.angle_max - usable_range / 2) / laser_range * ranges.shape[0])
        angles = angles[skip_left:-1 - skip_right]
        ranges = ranges[skip_left:-1 - skip_right]

    inf_mask = np.isinf(ranges)
    if inf_mask.any():
        ranges = ranges[~inf_mask]
        angles = angles[~inf_mask]

    points = np.zeros((ranges.shape[0], 2))
    points[:, 0] = -np.sin(angles) * ranges
    points[:, 1] = np.cos(angles) * ranges

    return points

In [4]:
def find_left_right_border(points, margin_relative=0.1):
    margin = int(points.shape[0] * margin_relative)

    relative = points[margin + 1:-margin, :] - points[margin:-margin - 1, :]
    distances = np.linalg.norm(relative, axis=1)

    return margin + np.argmax(distances) + 1

In [131]:
def handle_scan(laser_scan, delta_time):
    points = get_scan_as_cartesian(laser_scan)

    if points.shape[0] == 0:
        print("Skipping current laser scan message since it contains no finite values.")
        return

    split = find_left_right_border(points)

    right_wall = points[:split:4, :]
    left_wall = points[split::4, :]

    left_circle = Circle.fit(left_wall)
    right_circle = Circle.fit(right_wall)

    barrier_start = int(points.shape[0] * (0.5 - parameters.barrier_size_realtive))
    barrier_end = int(points.shape[0] * (0.5 + parameters.barrier_size_realtive))
    barrier = np.max(points[barrier_start: barrier_end, 1])

    print("follow_walls(%s, %s, %s, %s)" % (left_circle, right_circle, barrier, delta_time))

    print("show_circle_in_rviz(%s, %s, %s)" % (left_circle, left_wall, 0))
    print("show_circle_in_rviz(%s, %s, %s)" % (right_circle, right_wall, 1))

In [132]:
# lastly start the wallfollowing
handle_scan(scan_message, delta_time)

follow_walls(<__main__.Circle object at 0x122b4d358>, <__main__.Circle object at 0x122b4d198>, 3.1684158950919716, 0)
show_circle_in_rviz(<__main__.Circle object at 0x122b4d358>, [[-0.48464823  1.07587916]
 [-0.47903671  1.01570315]
 [-0.4851907   0.9838694 ]
 [-0.50045622  0.97169578]
 [-0.51923168  0.96633714]
 [-0.52771096  0.94229572]
 [-0.53097778  0.91048264]
 [-0.50372884  0.83011944]
 [-0.51280356  0.81274446]
 [-0.55103469  0.84046758]
 [-0.54479517  0.80013887]
 [-0.53732217  0.76029328]
 [-0.52803952  0.72015504]
 [-0.52963224  0.69650175]
 [-0.51880073  0.65809559]
 [-0.50996049  0.62416047]
 [-0.51754531  0.61134921]
 [-0.51428972  0.58643503]
 [-0.52310138  0.5758897 ]
 [-0.52827611  0.56157404]
 [-0.51776051  0.53149603]
 [-0.51843534  0.51393076]
 [-0.5128784   0.49097425]
 [-0.52283764  0.48330614]
 [-0.52074806  0.46478541]
 [-0.52726521  0.45432077]
 [-0.52511899  0.43673686]
 [-0.5186232   0.41623912]
 [-0.52027373  0.40283896]
 [-0.51760999  0.38651771]
 [-0.527523

In [57]:
df_include = rosbag_pandas.bag_to_dataframe('samples/first_test/2020-01-15-10-54-28.bag', include=['/scan'])

NameError: name 'rosbag_pandas' is not defined